In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pdfplumber
import subprocess
from docx import Document
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [2]:
def read_files(file_path):
    def read_docx_file(file_path):
        # Load the document
        doc = Document(file_path)
        
        # Initialize a list to hold the document text
        full_text = []
        
        # Iterate through the paragraphs in the document
        for paragraph in doc.paragraphs:
            full_text.append(paragraph.text)
        
        return '\n'.join(full_text)
    
    def read_doc_file(file_path):
        try:
            
            result = subprocess.run(['antiword', file_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            if result.returncode != 0:
                raise Exception(f"Error reading file: {result.stderr}")
            
            return result.stdout
        except FileNotFoundError:
            print(file_path,"antiword is not installed or not found in the system PATH.")
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None
    
    def read_pdf_file(file_path):
        # Open the PDF file
        full_text = []
        print("hi")
        with pdfplumber.open(file_path) as pdf:
            for page_num, page in enumerate(pdf.pages):
                text = page.extract_text()
                full_text.append(text)
        return '\n'.join(full_text)
                
    path = os.path.splitext(file_path)
    ext = path[1]

    if ext == '.doc':
       # print(file_path,ext)
        return read_doc_file(file_path)
    elif ext == '.docx':
        #print(file_path,ext)
        return read_docx_file(file_path)
    elif ext == '.pdf':
        return read_pdf_file(file_path)


def get_prefix(filename):
    """Extract the part of the filename up to the first underscore."""
    parts = filename.split('_')
    #print(parts[0])
    return parts[0]


In [3]:
file_names = []
file_content = []
categories = []
## Get all the file names under "Resume" and keep a list of all file paths
for root, dirs, files in os.walk('Resumes'):
    for file in files:
        file_path = os.path.join(root, file)
        file_names.append(file_path)
        file_content.append(read_files(file_path))
        print(root,"---", file, "---",file_path)
        # Infer Resume category
        if root == 'Resumes':
            #if 'Internship' in file: 
            #    categories.append("Internship")
            #else:
                categories.append("React Developer") 
        elif root == 'Resumes\workday resumes':
                categories.append("workday")
        elif root == 'Resumes\SQL Developer Lightning insight':
                categories.append("SQL Developer")
        elif root == 'Resumes\Peoplesoft resumes':
           # pref=get_prefix(file)
           # categories.append(pref)
            categories.append("Peoplesoft")
        else:
            categories.append("none") 
            

Resumes --- Internship_Ravali_Musquare Technologies (1).docx --- Resumes\Internship_Ravali_Musquare Technologies (1).docx
Resumes --- Internship_Susovan Bag_Musquare Technologies.docx --- Resumes\Internship_Susovan Bag_Musquare Technologies.docx
Resumes --- React Developer_Deepakreddy.docx --- Resumes\React Developer_Deepakreddy.docx
Resumes --- React Developer_Haripriya.docx --- Resumes\React Developer_Haripriya.docx
Resumes --- React Developer_Kamalakar Reddy.docx --- Resumes\React Developer_Kamalakar Reddy.docx
Resumes --- React Developer_Naveen sadhu.docx --- Resumes\React Developer_Naveen sadhu.docx
Resumes\React Developer_PavasGoswami.doc antiword is not installed or not found in the system PATH.
Resumes --- React Developer_PavasGoswami.doc --- Resumes\React Developer_PavasGoswami.doc
Resumes --- React Developer_Pragnya.docx --- Resumes\React Developer_Pragnya.docx
Resumes --- React Developer_Sarala Madasu-converted.docx --- Resumes\React Developer_Sarala Madasu-converted.docx
Re

Resumes\workday resumes --- ChinnaSubbarayuduM_Hexaware.docx --- Resumes\workday resumes\ChinnaSubbarayuduM_Hexaware.docx
Resumes\workday resumes --- Gopi Krishna_Hexaware.docx --- Resumes\workday resumes\Gopi Krishna_Hexaware.docx
Resumes\workday resumes\Hari Krishna M_Hexaware.doc antiword is not installed or not found in the system PATH.
Resumes\workday resumes --- Hari Krishna M_Hexaware.doc --- Resumes\workday resumes\Hari Krishna M_Hexaware.doc
Resumes\workday resumes\Harikrishna Akula_Hexaware.doc antiword is not installed or not found in the system PATH.
Resumes\workday resumes --- Harikrishna Akula_Hexaware.doc --- Resumes\workday resumes\Harikrishna Akula_Hexaware.doc
Resumes\workday resumes\Hima Mendu_Hexaware.doc antiword is not installed or not found in the system PATH.
Resumes\workday resumes --- Hima Mendu_Hexaware.doc --- Resumes\workday resumes\Hima Mendu_Hexaware.doc
Resumes\workday resumes --- Himaja G_(Hexaware).docx --- Resumes\workday resumes\Himaja G_(Hexaware).d

In [4]:
data = { "resume_name":file_names, "resume_content": file_content, "category":categories }

df = pd.DataFrame(data)
df.head()

,resume_name,resume_content,category
0,Resumes\Internship_Ravali_Musquare Technologie...,Name: Ravali P \n\n ...,React Developer
1,Resumes\Internship_Susovan Bag_Musquare Techno...,\nSUSOVAN BAG \nSeeking a challenging ...,React Developer
2,Resumes\React Developer_Deepakreddy.docx,Kanumuru Deepak Reddy\n\n\n\nCAREER OBJECTIVE:...,React Developer
3,Resumes\React Developer_Haripriya.docx,HARIPRIYA BATTINA \nExperience as UI Developer...,React Developer
4,Resumes\React Developer_Kamalakar Reddy.docx,KAMALAKAR REDDY. A \nLinked In: https://www.li...,React Developer


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   resume_name     79 non-null     object
 1   resume_content  53 non-null     object
 2   category        79 non-null     object
dtypes: object(3)
memory usage: 2.0+ KB


In [6]:
df.isna().sum()

resume_name        0
resume_content    26
category           0
dtype: int64

In [7]:
df_cleaned = df.dropna(subset=['resume_content'])

In [8]:
df_cleaned.isna().sum()

resume_name       0
resume_content    0
category          0
dtype: int64

In [9]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 78
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   resume_name     53 non-null     object
 1   resume_content  53 non-null     object
 2   category        53 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB


In [10]:
df_cleaned

,resume_name,resume_content,category
0,Resumes\Internship_Ravali_Musquare Technologie...,Name: Ravali P \n\n ...,React Developer
1,Resumes\Internship_Susovan Bag_Musquare Techno...,\nSUSOVAN BAG \nSeeking a challenging ...,React Developer
2,Resumes\React Developer_Deepakreddy.docx,Kanumuru Deepak Reddy\n\n\n\nCAREER OBJECTIVE:...,React Developer
3,Resumes\React Developer_Haripriya.docx,HARIPRIYA BATTINA \nExperience as UI Developer...,React Developer
4,Resumes\React Developer_Kamalakar Reddy.docx,KAMALAKAR REDDY. A \nLinked In: https://www.li...,React Developer
5,Resumes\React Developer_Naveen sadhu.docx,\nNaveen Sadhu\n\n\nTitle: software developer\...,React Developer
7,Resumes\React Developer_Pragnya.docx,\nPRAGNYA PATTNAIK\n \n \n \n Expertise: \n \...,React Developer
8,Resumes\React Developer_Sarala Madasu-converte...,"\n\n 204,Sri geethika prestige,road number 10...",React Developer
9,Resumes\React Developer_Thirupathiamma.docx,\n\nThirupathamma Balla\n\nSUMMARY:\n\n2.8 yea...,React Developer
11,Resumes\React Dev_Krishna Kanth.docx,\nUi-Developer/ React JS Developer \nNAME: KRI...,React Developer


In [11]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

#nltk.download('stopwords')
#nltk.download('wordnet')

# Initialize tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
# Function for preprocessing text
def preprocess_text(text):
    # Lowercase
    text = text.lower()

    # Remove special characters and digits
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenize
    tokens = text.split()

    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Join back into a single string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [13]:
df_cleaned['resume_content'] = df_cleaned['resume_content'].apply(preprocess_text)

X = df_cleaned['resume_content']
y = df_cleaned['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

C:\Users\hp\AppData\Local\Temp\ipykernel_17376\183222679.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['resume_content'] = df_cleaned['resume_content'].apply(preprocess_text)


In [14]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),          # Step 1: Convert text to numerical format
    ('classifier', MultinomialNB())        # Step 2: Classify with Naive Bayes
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

accuracy = pipeline.score(X_test, y_test)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 1.00


In [15]:
import pickle

with open('resume_classification_pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)